In [1]:
%%time
import math
import torch

from botorch.test_functions import SixHumpCamel
from botorch.fit import fit_gpytorch_mll
from botorch.models import SingleTaskGP
from botorch.utils.transforms import standardize, normalize
from gpytorch.mlls import ExactMarginalLogLikelihood
from botorch.acquisition import (
    ExpectedImprovement,
    ProbabilityOfImprovement,
    qMaxValueEntropy,
)
import matplotlib.pyplot as plt
from botorch.optim import optimize_acqf
from botorch.acquisition import qNoisyExpectedImprovement, qProbabilityOfImprovement
from time import time
import cProfile, io, pstats
from pstats import SortKey
import scipy

SMOKE_TEST = True

torch.manual_seed(123456)
torch.set_default_dtype(torch.double)

bounds = torch.tensor(SixHumpCamel._bounds).T
bounds_norm = torch.tensor([[0.0, 0.0], [1.0, 1.0]])
train_X = bounds[0] + (bounds[1] - bounds[0]) * torch.rand(5, 2)
train_Y = SixHumpCamel(negate=True)(train_X).unsqueeze(-1)

train_X = normalize(train_X, bounds=bounds)
train_Y = standardize(train_Y + 0.05 * torch.randn_like(train_Y))

model = SingleTaskGP(train_X, train_Y)
mll = ExactMarginalLogLikelihood(model.likelihood, model)
fit_gpytorch_mll(mll, max_attempts=10)

from botorch.acquisition.max_value_entropy_search import qLowerBoundMaxValueEntropy

candidate_set_size = 1000 if not SMOKE_TEST else 5
candidate_set = torch.rand(
    candidate_set_size, bounds_norm.size(1), device=bounds.device, dtype=bounds.dtype
)
qGIBBON = qLowerBoundMaxValueEntropy(model, candidate_set)
scipy.__version__

CPU times: user 4.05 s, sys: 3.32 s, total: 7.37 s
Wall time: 14.4 s


'1.7.3'

In [3]:
%%time

# prep different acqusition functions
candidate_set = torch.rand(
    10000, bounds.size(1), device=bounds.device, dtype=bounds.dtype
)
acqs = {}
# acqs["aa"] = qLowerBoundMaxValueEntropy(model, candidate_set)
# acqs["MES"] = qMaxValueEntropy(model, candidate_set)
acqs["EI"] = qNoisyExpectedImprovement(model, train_X)
# acqs["PI"] = qProbabilityOfImprovement(model, best_f=train_Y.max())


# eval and maximise acq functions
with cProfile.Profile() as pr:
    for acq in acqs.values():
        optimize_acqf(
            acq_function=acq,
            bounds=bounds_norm,
            q=3,
            num_restarts=5,
            raw_samples=100,
            sequential=True,
        )

sortby = SortKey.CUMULATIVE
ps = pstats.Stats(pr).sort_stats(sortby)
ps.print_stats()

scipy.__version__

CPU times: user 166 ms, sys: 57.7 ms, total: 224 ms
Wall time: 188 ms


'1.7.3'